In [1]:
import os
import sys

sys.path.append(
    os.path.join("..","Preprocesamiento")
)
sys.path.append(
    os.path.join("..","Models")
)
sys.path.append(
    os.path.join("..","Train_utils")
)

import torch
from torch import nn
import numpy as np
import torchvision
from torchvision import transforms

In [2]:
import dataLoader
from dataLoader import belugaDataset
from Custom_Transforms import *
dataset=belugaDataset(
    csv_file=os.path.join("..","metadata.csv"),
    im_folder=os.path.join("..","images")
)
dataset[0].keys()

dict_keys(['x', 'date', 'y', 'vp'])

In [3]:
from VAE_NETs.P_NET import P_NET
from VAE_NETs.Q_NET import Q_NET
from pyro_VAE import Flexible_Encoding_Decoding_VAE
from Encoding_Decoding_modules.Basic_Encoding_Decoding_Module import Basic_Convolutional_EDM

In [37]:
CEDM=Basic_Convolutional_EDM(
    repr_sizes=[3,3,3,3,3,3,6,6,6,9],
    #repr_sizes=[3,3,3,3,3,3,4,4,4,4,4,5,5,5],
    #repr_sizes=[3,6],
    kernel_size=11,
    activators=nn.LeakyReLU(),
    pooling=False,
    batch_norm=False,
    dropout=None,
    stride=1
)

p=P_NET(
    layer_size=[18900,1000,500,250][::-1],
    #layer_size=[342720,1000,500,250][::-1],
    dec_activators=nn.ReLU()
)
q=Q_NET(
    layer_size=[18900,1000,500,250],
    #layer_size=[342720,1000,500,250],
    enc_activators=nn.ReLU()
)

In [21]:
VAE=Flexible_Encoding_Decoding_VAE(
    encoding_decoding_module=CEDM,
    P_NET=p,
    Q_NET=q,
    losses_weigths={"generative_loss":1},
    subsample=None,
    sig_scale=1
)

In [22]:
Ct=transforms.Compose([
    vp_one_hot_encoding(),
    MultiInputResize(),
    MultiInputToTensor(),
    SelectInput(output_keys=["x","vp"])
])

In [23]:
Ct(dataset[0])["x"].shape

torch.Size([3, 300, 100])

In [38]:
print(CEDM.sanity_check(Ct(dataset[0])["x"].unsqueeze(0)))
CEDM.DEC(CEDM.ENC(Ct(dataset[0])["x"].unsqueeze(0))).shape

torch.Size([1, 18900])


torch.Size([1, 3, 300, 100])

In [44]:
VAE.eval()
VAE.forward_pass(Ct(dataset[0])["x"].unsqueeze(0))

(tensor([[9.4111e-03, 1.8626e-02, 1.6648e-02, 3.7308e-02, 2.3556e-02, 2.4564e-02,
          0.0000e+00, 3.4814e-02, 0.0000e+00, 2.7635e-02, 1.1529e-02, 0.0000e+00,
          0.0000e+00, 1.7640e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 4.6358e-02, 2.3339e-02, 1.1718e-03, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 6.4589e-03, 0.0000e+00, 1.0638e-03,
          3.0155e-02, 0.0000e+00, 3.2995e-02, 2.2640e-02, 1.0918e-03, 0.0000e+00,
          0.0000e+00, 4.3145e-04, 0.0000e+00, 0.0000e+00, 3.6735e-02, 0.0000e+00,
          1.1696e-02, 1.2491e-02, 2.7748e-02, 0.0000e+00, 0.0000e+00, 4.0952e-02,
          4.0950e-02, 2.0410e-02, 1.2749e-02, 0.0000e+00, 2.5095e-02, 0.0000e+00,
          0.0000e+00, 6.2928e-04, 0.0000e+00, 0.0000e+00, 4.4895e-03, 1.2091e-02,
          7.5259e-04, 2.2456e-02, 0.0000e+00, 2.1126e-02, 2.1261e-02, 0.0000e+00,
          1.6905e-02, 2.9984e-03, 1.9383e-02, 1.1355e-02, 0.0000e+00, 5.8404e-03,
          9.2720

In [46]:
[616896,1000,500,250][::-1]

[250, 500, 1000, 616896]

In [49]:
np.unique(dataset.df["whale_id"].values)

array(['whale000', 'whale001', 'whale002', 'whale003', 'whale004',
       'whale005', 'whale006', 'whale007', 'whale008', 'whale009',
       'whale010', 'whale011', 'whale012', 'whale013', 'whale014',
       'whale015', 'whale016', 'whale017', 'whale018', 'whale019',
       'whale020', 'whale021', 'whale022', 'whale023', 'whale024',
       'whale025', 'whale026', 'whale027', 'whale028', 'whale029',
       'whale030', 'whale031', 'whale032', 'whale033', 'whale034',
       'whale035', 'whale036', 'whale037', 'whale038', 'whale039',
       'whale040', 'whale041', 'whale042', 'whale043', 'whale044',
       'whale045', 'whale046', 'whale047', 'whale048', 'whale049',
       'whale050', 'whale051', 'whale052', 'whale053', 'whale054',
       'whale055', 'whale056', 'whale057', 'whale058', 'whale059',
       'whale060', 'whale061', 'whale062', 'whale063', 'whale064',
       'whale065', 'whale066', 'whale067', 'whale068', 'whale069',
       'whale070', 'whale071', 'whale072', 'whale073', 'whale0

In [40]:
int(len(dataset)*0.8)

4721